## LLM API 비교
- 비용으로 1차적으로 추린 후에 2차적으로 성능 비교

### Gemini API
- 다른 LLM 라인업 대비 항상 좀 더 저렴한 가격에 제공

In [1]:
# 환경변수에 GOOGLE_API_KEY 추가 필요
# ~/.zshrc에 export GOOGLE_API_KEY='<API KEY>'
import google.generativeai as genai
import os

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

model = genai.GenerativeModel('gemini-1.5-flash')

In [2]:
response = model.generate_content("Hi!")
print(response.text)

Hi! How can I help you today? 



### Claude API
- OpenAI에 필적하는 성능
  - 2024년 6월 기준 가장 좋은 성능의 모델 = Claude 3.5 Sonnet
- 현재 기준 가장 비싼 모델인 Claude 3 Opus 보유

In [3]:
# 환경변수에 ANTHROPIC_API_KEY 추가 필요
# ~/.zshrc에 export ANTHROPIC_API_KEY='<API KEY>'
import os

import anthropic

client = anthropic.Anthropic(
    api_key=os.environ['ANTHROPIC_API_KEY']
)

message = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=1000,
    temperature=0.0,
    system="Respond only in Yoda-speak.",
    messages=[
        {"role": "user", "content": "Hi!"}
    ]
)

print(message.content[0].text)

Greetings, young one. How may I assist you today?


### 동일한 Prompt에 대해 Input / Output 계산
- 비용 계산을 위해서는 Tokens 수를 알아야 함

In [4]:
from eval import get_eval_data

conversations = get_eval_data()

In [5]:
print(conversations[0])

P01: 맞다요즘 싸이월드 새로 생겼다매
P02: 나 싸이월드 복구하고 싶다
P01: 웅 키키귀여우뉴사진
P02: 진짜 추억돋네
P01: 짱많아 옛날 사진들
P01: 맨날 퍼가요~ 이거 했자나
P02: 맞아 싸이월드 미니미
P01: 추억 돋아서 너무조아
P02: 꾸미는거
P01: 마자
P02: 재밌었는데
P01: 귀여워 도토리충전
P02: 네이트온도 하고
P02: 도토리도 환불해준대자나
P01: 마자 키키개웃겨
P01: 요즘 사회적으로 멀 자꾸하나바
P02: 대박이야 진짜
P02: 그니깐 키키
P01: 아니 나오늘 엄마가 뭐 신청해달라그래서
P02: 웅
P01: 지원금 ? 신청함
P01: 진짜 요즘 지원금 엄청 많이 받았어
P02: 와 대박
P01: 뭔지는 잘 모르는데 이것저것 지원해주는거 많아서 좋은듯
P02: 다행이다
P01: 국가제도가 진짜 괜차나
P01: 학교에서도
P02: 두분다
P01: 맨날 장학금 해준다고
P02: 일을 못하는
P01: 이것저것 지원해주자나
P02: 상황이셔서 더
P01: 마자ㅠㅠ
P02: 도움주시고
P01: 나 그래서 진짜 그런거로
P02: 다행이다그래도
P01: 돈 마니받앗어
P02: 와대박


Gemini Pro & Flash
- 506 / 116 / 622

In [37]:
prompt = f"""아래 사용자 대화에 대해 3문장 내로 요약해주세요:

{conversations[0]}"""

model = genai.GenerativeModel('gemini-1.5-flash-001')
response = model.generate_content(prompt)
print(response.text)
response.usage_metadata

두 사람은 싸이월드 부활에 대한 추억을 공유하며 옛날 사진과 미니미, 도토리 등을 떠올립니다. 또한, 최근 사회적으로 지원금이 많아졌다는 이야기를 나누며, 학교에서도 장학금 등 다양한 지원을 받을 수 있어 다행이라고 생각합니다. 특히, 어려운 상황에 있는 사람들에게 도움이 되는 제도가 많아져서 긍정적인 반응을 보입니다. 



prompt_token_count: 506
candidates_token_count: 116
total_token_count: 622

모델 호출해보기 전에 Prompt가 몇 토큰인 지 미리 알 수 있음

In [38]:
# 실제로는 여기에 token 1씩 더 추가됨
model.count_tokens(prompt)

total_tokens: 505

동일한 Prompt에 대해 Gemini Pro와 Gemini Flash 둘 다 506 입력 토큰 -> 동일한 Tokenizer를 사용하는 것을 알 수 있음
- 더 많은 단어를 익힌 (보통 더 큰) 모델의 경우 다른 Tokenizer를 써서 입력 토큰 수가 달라질 수 있음

In [40]:
prompt = f"""아래 사용자 대화에 대해 3문장 내로 요약해주세요:

{conversations[0]}"""

model = genai.GenerativeModel('gemini-1.5-pro-001')
response = model.generate_content(prompt)
print(response.text)
response.usage_metadata

두 사람은 과거 싸이월드를 추억하며 미니미, 도토리 등을 회상하며 즐거워합니다. 대화 중에 최근 정부와 학교에서 지원금과 장학금을 많이 받았다는 내용이 나오고, 이에 대해 감사하며 국가 제도에 대한 긍정적인 평가를 내립니다. 전반적으로 과거 추억과 현재의 긍정적인 상황을 공유하며 가볍고 즐거운 분위기입니다. 



prompt_token_count: 506
candidates_token_count: 116
total_token_count: 622

Anthropic Claude
- 637 / 143 / 780

In [49]:
import os

import anthropic

client = anthropic.Anthropic(
    api_key=os.environ['ANTHROPIC_API_KEY']
)

message = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=1000,
    temperature=0.0,
    messages=[
        {"role": "user", "content": prompt}
    ]
)

print(message.content[0].text)
message.usage

1. 두 사람은 과거 싸이월드 사용 경험을 회상하며 추억을 공유하고 있다.
2. 최근 정부와 학교에서 제공하는 다양한 지원금 혜택에 대해 이야기하고 있다.
3. 경제적 어려움을 겪고 있는 상황에서 이러한 지원이 도움이 되고 있다고 말하고 있다.


Usage(input_tokens=637, output_tokens=143)

In [50]:
message = client.messages.create(
    model="claude-3-sonnet-20240229",
    max_tokens=1000,
    temperature=0.0,
    messages=[
        {"role": "user", "content": prompt}
    ]
)

print(message.content[0].text)
message.usage

요약:

1) 싸이월드와 같은 과거 인기 커뮤니티 사이트에 대한 추억을 회상하며 그 시절의 재미있던 경험을 공유했습니다.

2) 최근 정부와 학교에서 제공하는 다양한 지원금 제도에 대해 이야기하며 본인도 여러 지원금을 받았다고 말했습니다. 

3) 어려운 상황에 처한 사람들을 위해 국가 차원의 지원 제도가 잘 마련되어 있어 다행이라고 평가했습니다.


Usage(input_tokens=637, output_tokens=200)

In [51]:
message = client.messages.create(
    model="claude-3-opus-20240229",
    max_tokens=1000,
    temperature=0.0,
    messages=[
        {"role": "user", "content": prompt}
    ]
)

print(message.content[0].text)
message.usage

1. 싸이월드가 새로 생겨서 옛날 추억을 되새기며 즐거워하는 두 사람의 대화이다.
2. 요즘 사회적으로 여러 지원금과 장학금 등 국가 제도의 혜택이 많다는 것에 만족해하고 있다.
3. 두 사람 모두 일을 못하는 상황이라 이러한 지원이 더욱 도움이 된다며 다행스러워하고 있다.


Usage(input_tokens=637, output_tokens=164)

OpenAI
- 622 / 177 (GPT 3.5, GPT-4)
- 441 / 76 (GPT-4o)

In [55]:
from openai import OpenAI


OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

client = OpenAI(api_key=OPENAI_API_KEY)

completion = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=[{'role': 'user', 'content': prompt}],
    temperature=0.0
)

print(completion.choices[0].message.content)
completion.usage

두 사용자는 싸이월드를 통해 추억을 공유하고, 최근 받은 지원금에 대해 이야기하며 국가제도에 대한 긍정적인 평가를 나누었습니다.


CompletionUsage(completion_tokens=69, prompt_tokens=622, total_tokens=691)

In [56]:
from openai import OpenAI


OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

client = OpenAI(api_key=OPENAI_API_KEY)

completion = client.chat.completions.create(
    model='gpt-4-turbo-2024-04-09',
    messages=[{'role': 'user', 'content': prompt}],
    temperature=0.0
)

print(completion.choices[0].message.content)
completion.usage

사용자들은 최근 재개된 싸이월드에 대해 이야기하며, 옛날 사진과 미니미 꾸미기 등 추억을 회상합니다. 또한, 도토리 충전과 같은 싸이월드의 특징적인 요소들에 대해 언급하며 재미있었던 경험을 공유합니다. 대화에서는 또한 최근 받은 지원금과 장학금에 대한 이야기도 나누며, 이러한 지원이 도움이 되고 있다고 언급합니다.


CompletionUsage(completion_tokens=177, prompt_tokens=622, total_tokens=799)

In [57]:
from openai import OpenAI


OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

client = OpenAI(api_key=OPENAI_API_KEY)

completion = client.chat.completions.create(
    model='gpt-4o-2024-05-13',
    messages=[{'role': 'user', 'content': prompt}],
    temperature=0.0
)

print(completion.choices[0].message.content)
completion.usage

사용자들은 싸이월드의 복구와 관련된 추억을 이야기하며, 옛날 사진과 미니미 꾸미기, 도토리 충전 등의 재미를 회상하고 있습니다. 또한, 최근 정부의 다양한 지원금 제도에 대해 이야기하며, 이를 통해 많은 도움을 받고 있다는 점을 긍정적으로 평가하고 있습니다.


CompletionUsage(completion_tokens=76, prompt_tokens=441, total_tokens=517)

Input Tokens 수 정리 (Prompt 고정)
- Gemini 506
- Claude 637
- GPT 622
- GPT-4o 441
  - Output은 어느 정도 조정이 가능해서 Input만 비교

한글 효율화 측면에서는 GPT-4o > Gemini > GPT-3.5/4 >= Claude 순
- 다른 Prompt에서는 조금씩 다를 수 있으나, 경향이 크게 달라지지 않을 것으로 예상

### 비용 계산
- Input Tokens는 바로 위에 수치 사용하고, Output Tokens는 가장 많은 Claude를 200 Tokens로 두고 비례해서 계산
  - ex. Gemini 159 Tokens
- 호출 횟수 별 가격표
  - https://docs.google.com/spreadsheets/d/1L4jfa8F7ifSbp779pInSRkT_JFnT57xsmJvOhvePOk4/edit?usp=sharing

모델 별 가격 (1M Token, 2024.06)
- Claude 3 Opus $15.00 / $75.00
- Claude 3 Sonnet $3.00 / $15.00
- Claude 3 Haiku $0.25 / $1.25
- Gemini Pro  $3.5 / $10.5 (일정 요청량까지 무료)
- Gemini Flash $0.35 / $1.05 (일정 요청량까지 무료)
- GPT-4o $5 / $15
- GPT-4 $10 / $30
- GPT-3.5 $0.5 / $1.5

1회 호출 비용
- Claude 3 Opus  $0.024 / 33.89원
- Claude 3 Sonnet $0.004 / 6.78원
- Claude 3 Haiku $0.0004 / 0.56원
- Gemini Pro $0.003	/ 4.75원
- Gemini Flash $0.0003 / 0.47원
- GPT-4o $0.004	/ 5.91원
- GPT-4 $0.012 / 16.67원
- GPT-3.5 $0.0006 / 0.83원

10000회 호출 비용
- Claude 3 Opus  $245.55 / 338,859원
- Claude 3 Sonnet $49.11 / 67,772원
- Claude 3 Haiku $4.09 / 5,648원
- Gemini Pro $34.39	/ 47,460원
- Gemini Flash $3.44 / 4,746원
- GPT-4o $42.82	/ 59,091원
- GPT-4 $120.79 / 166,686원
- GPT-3.5 $6.04 / 8,334원

100만회 호출 비용
- Claude 3 Opus  $24,555 / 33,885,900원
- Claude 3 Sonnet $4,911 / 6,777,180원
- Claude 3 Haiku $409.25 / 564,765원
- Gemini Pro $3,439.13 / 4,746,002원
- Gemini Flash $343.91 / 474,600원
- GPT-4o $4,281.92 / 5,909,054원
- GPT-4 $12,078.71 / 16,668,624원
- GPT-3.5 $603.94 / 833,431원

5000만회 호출 비용
- Claude 3 Opus  $1,227,750 / 1,694,295,000원
- Claude 3 Sonnet $245,550 / 338,859,000원
- Claude 3 Haiku $20,462.50 / 28,238,250원
- Gemini Pro $171,956.59 / 237,300,099원
- Gemini Flash $17,195.66 / 23,730,010원
- GPT-4o $214,096.15 / 295,452,692원
- GPT-4 $603,935.64 / 833,431,177원
- GPT-3.5 $30,196.78 / 41,671,559원